In [1]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.metrics import MeanSquaredError
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

2022-11-23 21:40:47.373171: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 21:40:47.796302: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-23 21:40:47.924571: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-23 21:40:49.132064: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
def build_model(number_of_feature):
    model = Sequential()
    model.add(LSTM(128, return_sequences=False,
              input_shape=(number_of_feature, 1)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64))
    model.add(Dropout(0.2))
    model.add(Dense(32))
    model.add(Dropout(0.2))
    # model.add(LSTM(50, return_sequences=False))
    model.add(Dense(1, activation='linear'))

    model.compile(optimizer='adam',
                  loss='mse', metrics=[MeanSquaredError()])
    return model

In [3]:
def read_raw_dataset(url: str, index_col: str, target_name: str, drop = []):
    dataset = pd.read_csv(url, index_col=index_col)
    drop_cols = drop + [target_name]
    X_raw = dataset.drop(labels=drop_cols, axis=1)
    y = dataset[target_name]
    return X_raw, y

In [4]:
def select_features(X: pd.DataFrame, y: pd.Series, k=10):
	selector = SelectKBest(score_func=f_regression, k=k)
	selector.fit(X, y)
	return X[X.columns[selector.get_support(indices=True)]], selector

read dataset of each chunk and split it into raw features and target

In [5]:
def getDataset(id): 
    return read_raw_dataset(
    "datasets/chunk{}.csv".format(id), index_col="ID", target_name="VMOS")

# pd.read_csv("datasets/chunk{}.csv".format(id), index_col="ID")
raw_datasets = list(map(getDataset, range(10)))


In [6]:
def select(dataset): 
    return select_features(dataset[0], dataset[1])[0]

Xs = list(map(select, raw_datasets))


In [7]:
targets = list(map(lambda d : d[1], raw_datasets))
train_test_sets = list(map(train_test_split, Xs, targets))

In [8]:
def train(train_test_set):
    X_train, X_test, y_train, y_test = train_test_set
    lstm = build_model(10)
    lstm.fit(X_train, y_train, validation_split=0.05,
             batch_size=128, epochs=100, shuffle=True)
    lstm_pred = lstm.predict(X_test)
    return lstm.get_weights(), mean_squared_error(y_test, lstm_pred), X_train.shape[0]


In [9]:
train_results = list(map(train, train_test_sets))

2022-11-23 21:40:52.012532: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-23 21:40:52.060549: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-23 21:40:52.061133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-23 21:40:52.064489: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow 

Epoch 1/100


2022-11-23 21:40:56.106893: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


 1/50 [..............................] - ETA: 2:56 - loss: 12.0263 - mean_squared_error: 12.0263

2022-11-23 21:40:57.257786: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


50/50 [==============================] - 4s 13ms/step - loss: 1.3135 - mean_squared_error: 1.3135 - val_loss: 0.2323 - val_mean_squared_error: 0.2323
Epoch 2/100
50/50 [==============================] - 0s 7ms/step - loss: 0.5547 - mean_squared_error: 0.5547 - val_loss: 0.1553 - val_mean_squared_error: 0.1553
Epoch 3/100
50/50 [==============================] - 0s 7ms/step - loss: 0.4209 - mean_squared_error: 0.4209 - val_loss: 0.0355 - val_mean_squared_error: 0.0355
Epoch 4/100
50/50 [==============================] - 0s 7ms/step - loss: 0.3333 - mean_squared_error: 0.3333 - val_loss: 0.0077 - val_mean_squared_error: 0.0077
Epoch 5/100
50/50 [==============================] - 0s 7ms/step - loss: 0.3238 - mean_squared_error: 0.3238 - val_loss: 0.0162 - val_mean_squared_error: 0.0162
Epoch 6/100
50/50 [==============================] - 0s 8ms/step - loss: 0.2746 - mean_squared_error: 0.2746 - val_loss: 0.0025 - val_mean_squared_error: 0.0025
Epoch 7/100
50/50 [==========================

In [10]:
import utils

In [11]:
list_weights = list(map(lambda res : (res[0], res[2]), train_results))
weights = utils.aggregate(list_weights)

In [12]:
lstm_prime = build_model(10)
lstm_prime.set_weights(weights)

In [16]:
def prime_mse(train_test_set):
    X_test = train_test_set[1]
    y_test = train_test_set[3]
    y_pred = lstm_prime.predict(X_test)
    return mean_squared_error(y_test, y_pred)

In [14]:
for id, (_, mse, _) in enumerate(train_results):
    print("Chunk {} mse: {}".format(id, mse))

Chunk 0 mse: 0.0025605832781050706
Chunk 1 mse: 0.0014932798349250455
Chunk 2 mse: 0.00421649961706523
Chunk 3 mse: 0.002311584366245289
Chunk 4 mse: 0.002448758042278297
Chunk 5 mse: 0.001148440480187892
Chunk 6 mse: 0.00583611054174109
Chunk 7 mse: 0.004117049216737199
Chunk 8 mse: 0.003499718769633593
Chunk 9 mse: 0.0018360137456545232


In [17]:
prime_mses = list(map(prime_mse, train_test_sets))
for id, mse in enumerate(prime_mses):
    print("Chunk {} mse: {}".format(id, mse))

70/70 [==============================] - 0s 2ms/step
Chunk 0 mse: 4.664856088132856
Chunk 1 mse: 4.850547341229694
Chunk 2 mse: 4.405233181249873
Chunk 3 mse: 4.547756729457723
Chunk 4 mse: 4.588311194357214
Chunk 5 mse: 4.694681712671049
Chunk 6 mse: 4.982891103018054
Chunk 7 mse: 4.893054069091335
Chunk 8 mse: 4.7930134512123015
Chunk 9 mse: 4.832244269117969
